In [2]:
import tensorflow as tf
from gensim import models
from kerashypetune import KerasGridSearch
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")

In [21]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = vectorizer(np.array([[s] for s in train_x])).numpy()    
    val_x = vectorizer(np.array([[s] for s in val_x])).numpy()
    test_x = vectorizer(np.array([[s] for s in test_x])).numpy()

    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, 
                                       output_dim = emb_dim, 
                                       embeddings_initializer=keras.initializers.Constant(matrix),
                                       trainable=True)
    x = embedding_layer(inputs)
    x = layers.Bidirectional(layers.GRU(32, return_sequences = True, dropout=0.2))(x)
    x = layers.Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    x = layers.concatenate([avg_pool, max_pool])
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "logistic_models/{itr:04}".format(itr=cnt)
    checkpoint_path = model_path + "-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir logistic_models
    !mkdir $model_path
    
    model.compile(tf.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=[  "accuracy", 
                                                                                  tf.keras.metrics.AUC(name='auc')])
    

    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_auc', verbose=1,
            save_weights_only=True, save_best_only=True, mode='max')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6634 - accuracy: 0.5867 - auc: 0.6453
Epoch 00001: val_auc improved from -inf to 0.82117, saving model to logistic_models/0001-0001.ckpt
63/63 [==============================] - 28s 441ms/step - loss: 0.6634 - accuracy: 0.5867 - auc: 0.6453 - val_loss: 0.5538 - val_accuracy: 0.7370 - val_auc: 0.8212
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.2541 - accuracy: 0.9072 - auc: 0.9633
Epoch 00002: val_auc improved from 0.82117 to 0.92728, saving model to logistic_models/0001-0002.ckpt
63/63 [==============================] - 23s 368ms/step - loss: 0.2541 - accuracy: 0.9072 - auc: 0.9633 - val_loss: 0.3410 - val_accuracy: 0.8510 - val_auc: 0.9273
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.0220 - accuracy: 0.9949 - auc: 0.9993
Epoch 00003: val_auc improved from 0.92728 to 0.92816, saving model to logistic_models/0001-0003.ckpt
63/63 [==============================] - 22s 

In [22]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.40098280310630796 +/- 0.040388082240039126
test avg acc:  0.8566000103950501 +/- 0.011217836014866839
test avg auc:  0.9227044820785523 +/- 0.009739496141778187


val avg loss:  0.4076594650745392 +/- 0.03725215627636896
val avg acc:  0.8623999834060669 +/- 0.011909661759286502
val avg auc:  0.9225934147834778 +/- 0.0066072951739487235


train avg loss:  0.01120500632096082 +/- 0.010938692182990147
train avg acc:  0.9980000138282776 +/- 0.0019300288527991634
train avg auc:  0.9997920155525207 +/- 0.00020438760212209507
